In [1]:
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
import pandas as pd
import numpy as np

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('../data/veterans_lung_cancer.csv')
df.describe()

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Survival_label_lower_bound,Survival_label_upper_bound,Age_in_years,Karnofsky_score,Months_from_Diagnosis,Celltype=adeno,Celltype=large,Celltype=smallcell,Celltype=squamous,Prior_therapy=no,Prior_therapy=yes,Treatment=standard,Treatment=test
count,137.000000,137.0,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000
mean,121.627737,inf,58.306569,58.569343,8.773723,0.197080,0.197080,0.350365,0.255474,0.708029,0.291971,0.503650,0.496350
std,157.816722,NaN,10.541628,20.039592,10.612141,0.399253,0.399253,0.478835,0.437728,0.456337,0.456337,0.501821,0.501821
min,1.000000,1.0,34.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,25.0,51.000000,40.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,80.0,62.000000,60.000000,5.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,144.000000,177.0,66.000000,75.000000,11.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,999.000000,inf,81.000000,99.000000,87.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:

y_lower_bound = df['Survival_label_lower_bound']
y_upper_bound = df['Survival_label_upper_bound']
X = df.drop(['Survival_label_lower_bound', 'Survival_label_upper_bound'], axis=1)
y_lower_bound, y_upper_bound

(0       72.0
 1      411.0
 2      228.0
 3      126.0
 4      118.0
        ...  
 132    133.0
 133    111.0
 134    231.0
 135    378.0
 136     49.0
 Name: Survival_label_lower_bound, Length: 137, dtype: float64,
 0       72.0
 1      411.0
 2      228.0
 3      126.0
 4      118.0
        ...  
 132    133.0
 133    111.0
 134    231.0
 135    378.0
 136     49.0
 Name: Survival_label_upper_bound, Length: 137, dtype: float64)

In [4]:
X

,Age_in_years,Karnofsky_score,Months_from_Diagnosis,Celltype=adeno,Celltype=large,Celltype=smallcell,Celltype=squamous,Prior_therapy=no,Prior_therapy=yes,Treatment=standard,Treatment=test
0,69.0,60.0,7.0,0,0,0,1,1,0,1,0
1,64.0,70.0,5.0,0,0,0,1,0,1,1,0
2,38.0,60.0,3.0,0,0,0,1,1,0,1,0
3,63.0,60.0,9.0,0,0,0,1,0,1,1,0
4,65.0,70.0,11.0,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
132,65.0,75.0,1.0,0,1,0,0,1,0,0,1
133,64.0,60.0,5.0,0,1,0,0,1,0,0,1
134,67.0,70.0,18.0,0,1,0,0,0,1,0,1
135,65.0,80.0,4.0,0,1,0,0,1,0,0,1


In [5]:
rs = ShuffleSplit(n_splits=2, test_size=.2, random_state=42)
train_index, valid_index = next(rs.split(X))
len(train_index), len(valid_index)

(109, 28)

In [6]:
dtrain = xgb.DMatrix(X.values[train_index, :])
dtrain.set_float_info('label_lower_bound', y_lower_bound[train_index])
dtrain.set_float_info('label_upper_bound', y_upper_bound[train_index])

In [7]:

dvalid = xgb.DMatrix(X.values[valid_index, :])
dvalid.set_float_info('label_lower_bound', y_lower_bound[valid_index])
dvalid.set_float_info('label_upper_bound', y_upper_bound[valid_index])

In [8]:

params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.20,
          'max_depth': 5,
          'lambda': 0.01,
          'alpha': 0.02}

bst = xgb.train(params, dtrain, num_boost_round=10000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=50)

[0]	train-aft-nloglik:12.66083	valid-aft-nloglik:12.95680
[1]	train-aft-nloglik:11.91472	valid-aft-nloglik:12.24435
[2]	train-aft-nloglik:11.24083	valid-aft-nloglik:11.58992
[3]	train-aft-nloglik:10.63209	valid-aft-nloglik:11.00682
[4]	train-aft-nloglik:10.08223	valid-aft-nloglik:10.48380
[5]	train-aft-nloglik:9.58555	valid-aft-nloglik:10.00134
[6]	train-aft-nloglik:9.13669	valid-aft-nloglik:9.58304
[7]	train-aft-nloglik:8.73124	valid-aft-nloglik:9.19312
[8]	train-aft-nloglik:8.36476	valid-aft-nloglik:8.85334
[9]	train-aft-nloglik:8.03322	valid-aft-nloglik:8.54856
[10]	train-aft-nloglik:7.73358	valid-aft-nloglik:8.28207
[11]	train-aft-nloglik:7.46279	valid-aft-nloglik:8.03835
[12]	train-aft-nloglik:7.21896	valid-aft-nloglik:7.80253
[13]	train-aft-nloglik:6.99871	valid-aft-nloglik:7.58763
[14]	train-aft-nloglik:6.79919	valid-aft-nloglik:7.40082
[15]	train-aft-nloglik:6.61881	valid-aft-nloglik:7.22960
[16]	train-aft-nloglik:6.45582	valid-aft-nloglik:7.07631
[17]	train-aft-nloglik:6.30830

In [9]:
times_pred = bst.predict(dvalid, output_margin=True)
times_pred

array([1.2312315, 3.9123733, 3.9879122, 2.687117 , 4.0792427, 5.6348734,
       5.745776 , 3.2588155, 5.1970477, 4.494064 , 5.23722  , 2.4928775,
       5.31296  , 5.8244333, 3.7517586, 3.1658204, 3.8177469, 4.87757  ,
       5.7222567, 5.2050066, 5.0286746, 4.253947 , 4.9989595, 3.474886 ,
       3.260651 , 3.736475 , 3.7671022, 4.2022977], dtype=float32)

In [10]:
times_true = []
event_true = []
for idx in valid_index:
    if y_upper_bound[idx] is np.inf or not y_upper_bound[idx]:
        event = 0
    else:
        event = 1 
    event_true.append(event)
    times = y_lower_bound[idx]
    times_true.append(times)
times_true = np.array(times_true)
event_true = np.array(event_true)

In [11]:
times_true.shape

(28,)

In [12]:
from lifelines.utils import concordance_index
c_index = concordance_index(times_true, times_pred, event_true)
c_index

0.636604774535809